In [69]:
#library imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import talib
import warnings
from matplotlib.pyplot import figure
warnings.filterwarnings('ignore')

In [70]:
#data imports
volumes = pd.read_csv('Data/stock_volumes.csv', index_col = 'date', parse_dates = True)
prices = pd.read_csv('Data/stock_prices.csv', index_col = 'date', parse_dates = True)
info = pd.read_csv('Data/stock_info.csv')
sp_listings = pd.read_csv('Data/sp500_listings.csv', index_col = 'date', parse_dates = True)

## Data Cleaning

In [71]:
for col in prices.columns:
    #select the stock
    stock = prices[[col]]
    #find delist date
    delist_date = sp_listings.where(sp_listings['stock'] == col).last_valid_index() + pd.DateOffset(1)
    #set all values after delist date to NaN
    stock.loc[delist_date:][col] = np.nan
    #replace in original dataframe
    prices[col] = stock[col]

In [72]:
prices

,905270,921795,904261,905261,916328,923024,936365,902355,912215,929813,...,9889RL,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,22.75,12.63,30.00,29.61,3.8736,11.00,4.1250,24.50,58.00,19.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,22.56,12.57,31.50,29.37,3.8876,11.00,4.0000,24.50,57.75,18.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,22.38,12.41,32.25,28.89,3.8806,11.38,3.9375,24.63,57.88,18.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,21.81,12.60,32.13,28.41,3.8387,11.50,3.8125,24.75,57.50,18.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,22.25,12.49,32.13,29.13,3.8387,11.50,3.8125,25.00,57.75,19.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
drop_columns = []

for col in prices.columns:
    if prices[col].isna().value_counts()[1] == prices.shape[0]:
        drop_columns.append(col)

prices = prices.drop(columns = drop_columns)

In [74]:
returns = prices.copy(deep = True)
returns = prices_r.pct_change()*100

In [75]:
for col in returns.columns:
    #select the stock
    stock = prices_r[[col]]
    #find delist date
    delist_date = sp_listings.where(sp_listings['stock'] == col).last_valid_index() + pd.DateOffset(1)
    #set all values after delist date to NaN
    stock.loc[delist_date:][col] = np.nan
    #replace in original dataframe
    returns[col] = stock[col]

### Risk Adjusted Returns (Sharpe Ratio)

In [77]:
returns = returns/100
returns.head()

,905270,921795,904261,905261,916328,923024,936365,902355,912215,929813,...,9660J1,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
1990-01-03,-0.008352,-0.004751,0.050000,-0.008105,0.003614,0.000000,-0.030303,0.000000,-0.004310,-0.013158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,-0.007979,-0.012729,0.023810,-0.016343,-0.001801,0.034545,-0.015625,0.005306,0.002251,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,-0.025469,0.015310,-0.003721,-0.016615,-0.010797,0.010545,-0.031746,0.004872,-0.006565,-0.013333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,0.020174,-0.008730,0.000000,0.025343,0.000000,0.000000,0.000000,0.010101,0.004348,0.034054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-09,-0.011236,-0.029624,-0.004046,-0.010985,-0.003647,-0.010435,0.016393,0.000000,0.004329,-0.032933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
mean = returns.mean()
mean

905270    0.000538
921795   -0.000034
904261    0.000104
905261    0.000118
916328    0.000563
            ...   
131745    0.001479
69487D    0.001350
68157P    0.001450
9110RA    0.002653
292703    0.000747
Length: 1236, dtype: float64

In [79]:
std = returns.std()
std

905270    0.019287
921795    0.028428
904261    0.021595
905261    0.020969
916328    0.016001
            ...   
131745    0.033784
69487D    0.027612
68157P    0.024868
9110RA    0.039285
292703    0.021340
Length: 1236, dtype: float64

**$Sharpe Ratio = \frac{Return - Risk Free Rate}{Standard Deviation}$**

In [80]:
std = std*np.sqrt(252)
rfr = 0.02
std

905270    0.306168
921795    0.451281
904261    0.342813
905261    0.332869
916328    0.254000
            ...   
131745    0.536304
69487D    0.438324
68157P    0.394769
9110RA    0.623631
292703    0.338764
Length: 1236, dtype: float64

In [81]:
for col in prices.columns:
    cumu_return = (prices[col][-1]/prices[col][0])-1

In [82]:
# risk-free rate is a minimum return that you can expect from an investment
# rfr = Risk Free Rate = 2% per annum
sharpe = (returns - rfr)/std 
sharpe

,905270,921795,904261,905261,916328,923024,936365,902355,912215,929813,...,9660J1,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
1990-01-03,-0.092602,-0.054845,0.087511,-0.084434,-0.064511,-0.041038,-0.080989,-0.053221,-0.074800,-0.100036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,-0.091384,-0.072524,0.011113,-0.109182,-0.085829,0.029846,-0.057357,-0.039101,-0.054611,-0.060339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,-0.148511,-0.010392,-0.069195,-0.109997,-0.121249,-0.019401,-0.083312,-0.040256,-0.081738,-0.100565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,0.000569,-0.063664,-0.058341,0.016052,-0.078740,-0.041038,-0.032200,-0.026342,-0.048160,0.042401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-09,-0.102022,-0.109962,-0.070143,-0.093085,-0.093099,-0.062449,-0.005807,-0.053221,-0.048218,-0.159696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Processing

In [49]:
#drop very first row
prices_r = prices_r.iloc[1:, :]

In [60]:
#sample stock
stock = '905270'